## Kate Commitment Scheme 
Here we implement a Kate Commitment Scheme, proposed by [Aniket Kate, Gregory M. Zaverucha, and Ian Goldberg](https://www.iacr.org/archive/asiacrypt2010/6477178/6477178.pdf). This implemintation follows [KZG polynomial commitments](https://dankradfeist.de/ethereum/2020/06/16/kate-polynomial-commitments.html#fn:1) blog post. 

The scheme allows one to commit and proove commitment to secret polynomials. One can proove that they correctly evaluate the secret polynomial and did not try to trick the system by, for example, substituting the polynomial for another one. The proofs do not compromise the polynomial secrecy. This is very useful for SNARK systems, such as PLONK.

In [89]:
# Loading the BLS12-381 curve with Tate Pairing
load("bls-pairing.sage")

# The max degree of a supported polynomial, can be increased
POLY_SIZE = 100

### Trusted Setup 
We generate a set of secret random numbers (toxic waist), which we then convert to Elliptic Curve points. This is then used to work with polynomials. 

First, we generate a random number in `GF1` - field of the `BLS12-381`. We can also use it in the `GF2`| - field of the twisted `BLS12-381` curve. We  then use generators of each respective curves to calculate `g^x` for `x` in range from 0 to n. 

We could then use this same setup for all polynomials with degree smaller than the amount of points generated.

In [194]:
import random
# Set seed for reproduciability 
random.seed(0)

# Generate a random number in the G1F
toxic_random = G1F(1000)
#G1F.random_element() # !toxic secret!

# Generate nessessary points on the Elliptic Curves
poly_points_g1 = [G1 * (toxic_random ^ x) for x in range (0, POLY_SIZE)]
poly_points_g2 = [G2 * (toxic_random ^ x) for x in range (0, 2)]

# We must forget the toxic waste, otherwise someone can crack the system
del toxic_random

### Commit a Polynomial
Let us have a polynomial `T`, with degree under the max possible degree. 

To commit the polynomial, we evaluate the polynomial over the random element we have generated a step before. We do not have access to the point, so instead we use precomputed points generated on the previous step. We multiply these points by corresponding coeficients and sum together to get the final result, which is our commitment.

In [195]:
R.<t> = G1F['t'] # Setup polynomials inside of the G1F

# The selected polynomial we want to commit
polynomial = t^3 + t + 5

In [196]:
def commit_polynomial(polynomial):
    # Commit by submitting evaluation of a polynomial 
    commitment = evaluate_polynomial_as_point(polynomial)
    return commitment

In [197]:
# Evaluate the polynomial at a secret point using the notion of the secret cast to elliptic curve point
def evaluate_polynomial_as_point(polynomial):
    # Get a list of coeficients 
    polynomial_coef = polynomial.list()
    # Evaluate polynomial at each degree by multiplying the coeficient with the elliptic curve secret to the degree power
    single_degree_eval = [poly_points_g1[degree] * polynomial_coef[degree] for degree in range(0, polynomial.degree() + 1)]
    # Sum together all degree to get the final evaluation of the polynomial
    poly_eval = sum(single_degree_eval)
    return poly_eval

In [198]:
commitment = commit_polynomial(polynomial_T)

In [199]:
commitment

(2441088969857605299873535320891852548810597408768482629321138419663089805116082533435899184366955348706009278139181 : 3493865200853383019543064784559973558546332461238971727832233503353013461080532284222693016571825020864956376275043 : 1)

### Evaluate Polynomial 
Based on the value `z` from the `GF1` we calculate polynomial at this point, and output the proof that the calculation has been done correctly.

The proof consists of the polynomial `Q = (T - T(z)) / (t - z)` evaluated at the point `z`.

In [167]:
def prove_eval(polynomial, point):
    # Evaluate the polynomial at the point
    evaluation = polynomial(point)
    # Generate polynomial, that we will use for proof
    proof_polynomial = (polynomial - evaluation) / (t - point)
    assert proof_polynomial.is_integral(), "Polynomials must devide each other."
    proof_polynomial = proof_polynomial.numerator()
    # Generate proof by evaluating on toxic point using Reference String
    evaluation_proof = evaluate_polynomial_as_point(proof_polynomial)
    return  evaluation, evaluation_proof

In [168]:
# Select a random point to evaluate
point = G1F.random_element()
# Evaluate the polynomial at the point and generate the proof of evaluation
(evaluation, proof) = prove_eval(polynomial, point)

### Verify Proof
Verify that the committed polynomial was evaluated correctly at the specified point.

We use the fact, that `Q(s) * (s-z) = T(s) - T(z) = commitment - proof`. To check this, we use pairings and operate on elliptic curve points. We verify without knowing `s`.

In [169]:
def verify(point, evaluation, proof, commitment):
    # Precompute some elliptic curve points we will use in pairings
    secret_in_g2 = poly_points_g2[1]
    point_in_g2 = G2 * point
    evaluation_in_g1 = G1 * evaluation
    # Compute pairings 
    lhs = BLSpairing.ate_pair(proof, (secret_in_g2 - point_in_g2))
    rhs = BLSpairing.ate_pair(commitment - evaluation_in_g1, G2)
    print(lhs, '\n\n', rhs)
    # Check that two pairings match 
    assert lhs == rhs, "Evaluation check did not pass!"

In [170]:
verify(point, evaluation, proof, commitment)
print("Evaluation verification passed!")

3469352567491982967175112924294347174786959945520873797739519374708580172011540938457400093352253531571285851844519*w^11 + 2654533561021586457581644356961120757732790733256216629476026685613527485246797879613376226632650564174692513257823*w^10 + 2185074730013248348668162652438851729296493409397391876142706715954906434194177778501388160701577515776788243167030*w^9 + 3655835593377305282256906059318192490357685182205311730057783925694111876103740047799285665211314211786370844494319*w^8 + 2784904885023181055442577643768434984162276766426970248038770444043385345049116946171557125576797133006917758210624*w^7 + 40679776524632901444444167216105999546076034152613340469662477001520321253184894281709748285274592027321799616014*w^6 + 159450840804588648876352469533764446036167247712813350741489135316927053026913362991672025775735631816856026179018*w^5 + 1549275801391179399579982325956570782659808721437620760369409439747056506979867527696931206069980145620834441053624*w^4 + 3627529785459615988438973

AssertionError: Evaluation check did not pass!

In [164]:
assert(1 != 1, "tet")

In [137]:
poly_points_g2

[(3059144344244213709971259814753781636986470325476647558659373206291635324768958432433509563104347017837885763365758*u + 352701069587466618187139116011060144890029952792775240219908644239793785735715026873347600343865175952761926303160 : 927553665492332455747201965776037880757740193453592970025027978793976877002675564980949289727957565575433344219582*u + 1985150602287291935568054521177171638300868978215655730859378665066344726373823718423869104263333984641494340347905 : 1),
 (1386922592473214611742708812464665654657634003966034193122376603638067219219766503281046590491995997701369836416697*u + 1688553975710390430195250680142646612834759116623837370176813806683479800191844952882860581715038811064081134195619 : 1822679687313291224916541128555369277197992877040504991961962226633846276025901987460050516844615226061584774597146*u + 3000560883827790861607971193832101205904316732558065907416877659054157668342313160523199340695985181441270621870452 : 1),
 (548808907596218239094824026596377666